In [1]:
import re
import os
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
def get_concepts_from_file(file_path,file_name,source):
    
    list_of_concepts = list()
    
    file = open(file_path, 'r',encoding="utf8",errors = 'ignore') 
    Lines = file.readlines() 
    for line in Lines: 
        entry = line.strip()
        regular_exp_con = 'c="(.*)" ([0-9]*):([0-9]*) [0-9]*:([0-9]*)\|\|t="(.*)"'
        pattern_search = re.search(regular_exp_con, entry, re.IGNORECASE)

        if pattern_search:
            concept_dict = dict()
            concept_dict['source'] = source
            concept_dict['file_name'] = file_name 
            concept_dict['text'] = pattern_search.group(1)
            concept_dict['line_number'] = int(pattern_search.group(2))
            concept_dict['begin_word_num'] = int(pattern_search.group(3))
            concept_dict['end_word_num'] = int(pattern_search.group(4))
            concept_dict['concept_type'] = pattern_search.group(5)
            list_of_concepts.append(concept_dict)
    
    return list_of_concepts

In [3]:
data_file_path = os.path.dirname(os.getcwd()) + r'\Data\concept_assertion_relation_training_data' 
beth_file_path = data_file_path + r'\beth'
partners_file_path = data_file_path + r'\partners'

list_of_all_concepts = list()

for file in os.listdir(beth_file_path+r'\concept'):
    file_path = os.path.join(beth_file_path+r'\concept', file)
    list_of_all_concepts.extend(get_concepts_from_file(file_path,file.strip(".con"),'beth'))

for file in os.listdir(partners_file_path+r'\concept'):
    file_path = os.path.join(partners_file_path+r'\concept', file)
    list_of_all_concepts.extend(get_concepts_from_file(file_path,file.strip(".con"),'partners'))

In [4]:
concept_df = pd.DataFrame(list_of_all_concepts)

In [5]:
concept_df.head()

,source,file_name,text,line_number,begin_word_num,end_word_num,concept_type
0,beth,record-105,left basilar atelectasis,55,6,8,problem
1,beth,record-105,ventral hernia,143,1,2,problem
2,beth,record-105,htn,26,0,0,problem
3,beth,record-105,spontaneous echo contrast,68,1,3,problem
4,beth,record-105,cath,21,0,0,test


In [6]:
concept_df.groupby(['concept_type']).size()

concept_type
problem      7073
test         4608
treatment    4844
dtype: int64

In [7]:
def get_assertions_from_file(file_path,file_name,source):
    
    list_of_assertions = list()
    
    file = open(file_path, 'r',encoding="utf8",errors = 'ignore') 
    Lines = file.readlines() 
    
    for line in Lines: 
        entry = line.strip()
        regular_exp_con = 'c="(.*)" ([0-9]*):([0-9]*) [0-9]*:([0-9]*)\|\|t=".*"\|\|a="(.*)"'
        pattern_search = re.search(regular_exp_con, entry, re.IGNORECASE)

        if pattern_search:
            assertion_dict = dict()
            assertion_dict['source'] = source
            assertion_dict['file_name'] = file_name 
            assertion_dict['text'] = pattern_search.group(1)
            assertion_dict['line_number'] = int(pattern_search.group(2))
            assertion_dict['begin_word_num'] = int(pattern_search.group(3))
            assertion_dict['end_word_num'] = int(pattern_search.group(4))
            assertion_dict['assertion_type'] = pattern_search.group(5)
            list_of_assertions.append(assertion_dict)
    
    return list_of_assertions

In [8]:
list_of_all_assertions = list()

for file in os.listdir(beth_file_path+r'\ast'):
    file_path = os.path.join(beth_file_path+r'\ast', file)
    list_of_all_assertions.extend(get_assertions_from_file(file_path,file,'beth'))

for file in os.listdir(partners_file_path+r'\ast'):
    file_path = os.path.join(partners_file_path+r'\ast', file)
    list_of_all_assertions.extend(get_assertions_from_file(file_path,file,'partners'))

In [9]:
assertion_df = pd.DataFrame(list_of_all_assertions)

In [10]:
assertion_df.head()

,source,file_name,text,line_number,begin_word_num,end_word_num,assertion_type
0,beth,record-105.ast,left basilar atelectasis,55,6,8,present
1,beth,record-105.ast,ventral hernia,143,1,2,present
2,beth,record-105.ast,htn,26,0,0,present
3,beth,record-105.ast,spontaneous echo contrast,68,1,3,absent
4,beth,record-105.ast,80% lm lesion,21,6,8,present


In [11]:
assertion_df.groupby(['assertion_type']).size()

assertion_type
absent                          1596
associated_with_someone_else      89
conditional                       73
hypothetical                     382
possible                         309
present                         4624
dtype: int64

In [12]:
def get_relations_from_file(file_path,file_name,source):
    
    list_of_relations = list()
    
    file = open(file_path, 'r',encoding="utf8",errors = 'ignore') 
    Lines = file.readlines() 
    
    for line in Lines: 
        entry = line.strip()
        regular_exp_con = 'c="(.*)" ([0-9]*):([0-9]*) [0-9]*:([0-9]*)\|\|r="(.*)"\|\|c="(.*)" ([0-9]*):([0-9]*) [0-9]*:([0-9]*)'
        pattern_search = re.search(regular_exp_con, entry, re.IGNORECASE)

        if pattern_search:
            relation_dict = dict()
            relation_dict['source'] = source
            relation_dict['file_name'] = file_name 
            relation_dict['from_text'] = pattern_search.group(1)
            relation_dict['from_line_number'] = int(pattern_search.group(2))
            relation_dict['from_begin_word_num'] = int(pattern_search.group(3))
            relation_dict['from_end_word_num'] = int(pattern_search.group(4))
            relation_dict['relation_type'] = pattern_search.group(5)
            relation_dict['to_text'] = pattern_search.group(6)
            relation_dict['to_line_number'] = int(pattern_search.group(7))
            relation_dict['to_begin_word_num'] = int(pattern_search.group(8))
            relation_dict['to_end_word_num'] = int(pattern_search.group(9))
            list_of_relations.append(relation_dict)
    
    return list_of_relations

In [13]:
list_of_all_relations = list()

for file in os.listdir(beth_file_path+r'\rel'):
    file_path = os.path.join(beth_file_path+r'\rel', file)
    list_of_all_relations.extend(get_relations_from_file(file_path,file,'beth'))

for file in os.listdir(partners_file_path+r'\rel'):
    file_path = os.path.join(partners_file_path+r'\rel', file)
    list_of_all_relations.extend(get_relations_from_file(file_path,file,'partners'))

In [14]:
relation_df = pd.DataFrame(list_of_all_relations)

In [15]:
relation_df.head()

,source,file_name,from_text,from_line_number,from_begin_word_num,from_end_word_num,relation_type,to_text,to_line_number,to_begin_word_num,to_end_word_num
0,beth,record-105.rel,cath,21,0,0,TeRP,80% lm lesion,21,6,8
1,beth,record-105.rel,pefusion imaging,19,6,7,TeRP,perfusion defects,19,12,13
2,beth,record-105.rel,drugs,12,8,8,TrCP,known allergies,12,5,6
3,beth,record-105.rel,metal plate,26,7,8,TrAP,gsw,26,11,11
4,beth,record-105.rel,creams,145,14,14,TrNAP,any incisions,145,20,21


In [16]:
relation_df.groupby(['relation_type']).size()

relation_type
PIP      755
TeCP     166
TeRP     993
TrAP     885
TrCP     184
TrIP      51
TrNAP     62
TrWP      24
dtype: int64

In [ ]:
text = r"C:/Users/itsma/Documents/CS 6120 Project/Data/concept_assertion_relation_training_data/beth/txt/record-105.txt"

oFile = open(text, 'r')
line = oFile.read()

In [ ]:
line

In [ ]:
line.split("\n")[25].split()

In [ ]:
for file in tqdm(os.listdir(beth_file_path+r'\txt')):
    if(not file.endswith(".txt")):
        continue
    file_path = os.path.join(beth_file_path+r'\txt', file)
    file_name = file.strip(".txt")
    oFile = open(file_path, 'r')
    line = oFile.read()
    all_lines = line.split("\n")
    
    file_concepts = concept_df[(concept_df['file_name']==file_name)&(concept_df['source']=='beth')]
    
    for index,row in file_concepts.iterrows():
        words = all_lines[int(row['line_number'])-1].split()[int(row['begin_word_num']):int(row['end_word_num'])+1]
        if(" ".join(words).lower()!=row['text'].lower()):
            print(row)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

In [ ]:
inputs

In [ ]:
encodings = tokenizer.encode("Hello, my dog is cute",add_special_tokens = True)

In [ ]:
encodings

In [ ]:
input_ids = torch.tensor(encodings).long().unsqueeze(0)
        
outputs = model(input_ids,token_type_ids=None)

In [ ]:
len(outputs[0][0][5].data.numpy())

In [24]:
from BERT_utility import BERT_utility

In [21]:
from imp import reload

In [22]:
import BERT_utility

In [23]:
reload(BERT_utility)

<module 'BERT_utility' from 'C:\\Users\\itsma\\Documents\\CS 6120 Project\\CS6120\\Code\\BERT_utility.py'>

In [25]:
utility = BERT_utility()
#word_list = utility.process_string_finetune(line,0)

In [19]:
def create_pos_dict(concept):
    positions = dict()
    
    for index,row in concept.iterrows():
        for i in range(row['begin_word_num'],row['end_word_num']+1):
            positions[str(row['line_number'])+":"+str(i)] = row['concept_type']
    
    return positions

In [27]:
utility = BERT_utility()

all_words_list = list()
for file in tqdm(os.listdir(beth_file_path+r'\txt')):
    if(not file.endswith(".txt")):
        continue
    file_path = os.path.join(beth_file_path+r'\txt', file)
    file_name = file.strip(".txt")
    oFile = open(file_path, 'r')
    line = oFile.read()
    all_lines = line.split("\n")
    file_concepts = concept_df[(concept_df['file_name']==file_name)&(concept_df['source']=='beth')]
    positions = create_pos_dict(file_concepts)
    
    word_list = utility.process_string_finetune(line,0)
    
    for entry in word_list:
        key = str(entry['sentence_index'])+ ":" +str(entry['word_index'])
        if(key in positions):
            entry.update({"concept":positions[key]})
        else:
            entry.update({"concept":"blank"})
    
    all_words_list.extend(word_list)

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [06:11<00:00,  5.03s/it]


In [28]:
for file in tqdm(os.listdir(partners_file_path+r'\txt')):
    if(not file.endswith(".txt")):
        continue
    file_path = os.path.join(partners_file_path+r'\txt', file)
    file_name = file.strip(".txt")
    oFile = open(file_path, 'r')
    line = oFile.read()
    all_lines = line.split("\n")
    file_concepts = concept_df[(concept_df['file_name']==file_name)&(concept_df['source']=='partners')]
    positions = create_pos_dict(file_concepts)
    
    word_list = utility.process_string_finetune(line,0)
    
    for entry in word_list:
        key = str(entry['sentence_index'])+ ":" +str(entry['word_index'])
        if(key in positions):
            entry.update({"concept":positions[key]})
        else:
            entry.update({"concept":"blank"})
    
    all_words_list.extend(word_list)

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [05:27<00:00,  3.37s/it]


In [29]:
len(all_words_list)

149351

In [30]:
words_df = pd.DataFrame(all_words_list)

In [31]:
set(list(words_df['concept']))

{'blank', 'problem', 'test', 'treatment'}

In [ ]:
test_dict = {"a":"1", "b":"2"}

In [ ]:
if("a" in test_dict):
    print(test_dict["a"])

In [32]:
X = np.vstack(list(words_df["keyword_vector"]))                                
y = words_df["concept"]  

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
clf = LogisticRegression(random_state=0,solver="lbfgs",max_iter=1000).fit(X, y)

C:\Users\itsma\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [35]:
clf.score(X,y)

0.9940475791926402

In [36]:
test_data_file_path = os.path.dirname(os.getcwd()) + r'\Data\reference_standard_for_test_data' 

list_of_all_test_concepts = list()

for file in os.listdir(test_data_file_path+r'\concepts'):
    file_path = os.path.join(test_data_file_path+r'\concepts', file)
    list_of_all_test_concepts.extend(get_concepts_from_file(file_path,file.strip(".con"),'test_data'))

In [37]:
test_concept_df = pd.DataFrame(list_of_all_test_concepts)

In [38]:
test_concept_df.head()

,source,file_name,text,line_number,begin_word_num,end_word_num,concept_type
0,test_data,0001,his home regimen,111,8,10,treatment
1,test_data,0001,cad/chf,111,1,1,problem
2,test_data,0001,his acute gi bleed,111,20,23,problem
3,test_data,0001,basos,78,0,0,test
4,test_data,0001,biopsies,109,1,1,test


In [40]:
all_words_list_test = list()
test_data_texts_path =  os.path.dirname(os.getcwd()) + r'\Data\test_data'
for file in tqdm(os.listdir(test_data_texts_path)):
    if(not file.endswith(".txt")):
        continue
    file_path = os.path.join(test_data_texts_path, file)
    
    file_name = file.strip(".txt")
    
    oFile = open(file_path, 'r')
    
    line = oFile.read()
    
    all_lines = line.split("\n")
    
    file_concepts = test_concept_df[(test_concept_df['file_name']==file_name)&(test_concept_df['source']=='test_data')]
    
    positions = create_pos_dict(file_concepts)
    
    word_list = utility.process_string_finetune(line,0)
    
    for entry in word_list:
        key = str(entry['sentence_index'])+ ":" +str(entry['word_index'])
        if(key in positions):
            entry.update({"concept":positions[key]})
        else:
            entry.update({"concept":"blank"})
    
    all_words_list_test.extend(word_list)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [02:20<00:00,  4.68s/it]


In [41]:
test_word_df = pd.DataFrame(all_words_list_test)

In [42]:
X_test = np.vstack(list(test_word_df["keyword_vector"]))                                
y_test = test_word_df["concept"]  

In [43]:
clf.score(X_test,y_test)

0.9677156979290497

In [44]:
from sklearn.metrics import confusion_matrix

In [46]:
y_true = y_test
y_predict = clf.predict(X_test)

In [48]:
labels = ['blank', 'problem', 'test', 'treatment']

In [49]:
confusion_matrix(y_true,y_predict,labels)

array([[23757,   187,   135,   151],
       [  193,  3446,    17,    16],
       [  143,     7,  1968,    26],
       [  113,    32,    12,  1763]], dtype=int64)

In [50]:
from sklearn.metrics import f1_score

In [51]:
f1_score(y_true, y_predict, average='macro')

0.9374012222907829

In [ ]:
class_map = {"blank":0,"problem":1,"test":2,"treatment":3}

In [ ]:
encoding_list = list()
label_list = list()
utility = BERT_utility()

for file in tqdm(os.listdir(beth_file_path+r'\txt')):
    if(not file.endswith(".txt")):
        continue
    file_path = os.path.join(beth_file_path+r'\txt', file)
    file_name = file.strip(".txt")
    oFile = open(file_path, 'r')
    line = oFile.read()
    all_lines = line.split("\n")
    file_concepts = concept_df[(concept_df['file_name']==file_name)&(concept_df['source']=='beth')]
    positions = create_pos_dict(file_concepts)
    
    prior_sentence_index = -1
    
    word_list, encoding = utility.process_string_finetune(line,0)
    
    for i in range(len(all_lines)):
        labels = [0] * len(encoding[i])
        fil_word_list = [word for word in word_list if word["sentence_index"]==i+1] 
        for entry in fil_word_list:
            key = str(entry['sentence_index'])+ ":" +str(entry['word_index'])
            if(key in positions):
                for token_position in entry["bert_token_positions"]:
                    labels[token_position] = class_map[positions[key]]
        label_list.append(labels)
    
    encoding_list.extend(encoding)

for file in tqdm(os.listdir(partners_file_path+r'\txt')):
    if(not file.endswith(".txt")):
        continue
    file_path = os.path.join(partners_file_path+r'\txt', file)
    file_name = file.strip(".txt")
    oFile = open(file_path, 'r')
    line = oFile.read()
    all_lines = line.split("\n")
    file_concepts = concept_df[(concept_df['file_name']==file_name)&(concept_df['source']=='partners')]
    positions = create_pos_dict(file_concepts)
    
    prior_sentence_index = -1
    
    word_list, encoding = utility.process_string_finetune(line,0)
    
    for i in range(len(all_lines)):
        labels = [0] * len(encoding[i])
        
        fil_word_list = [word for word in word_list if word["sentence_index"]==i+1] 
        for entry in fil_word_list:
            key = str(entry['sentence_index'])+ ":" +str(entry['word_index'])
            if(key in positions):
                for token_position in entry["bert_token_positions"]:
                    labels[token_position] = class_map[positions[key]]
        label_list.append(labels)
    
    encoding_list.extend(encoding)

In [ ]:
[0]*5

In [ ]:
tokenizer.convert_ids_to_tokens([101, 2381, 1997, 2556, 7355, 1024, 102])

In [ ]:
label_list[87]

In [ ]:
encoding_list[87]

In [ ]:
len(encoding_list)

In [ ]:
import pickle

In [ ]:
pickle.dump(encoding_list,open("C:/Users/itsma/Documents/CS 6120 Project/input_ids.pkl","wb"))
pickle.dump(label_list,open("C:/Users/itsma/Documents/CS 6120 Project/label.pkl","wb"))

In [ ]:
finetuned_model = pickle.load(open("C:/Users/itsma/Documents/CS 6120 Project/CS6120/Model/finetuned_model.pkl","rb"))

In [ ]:
encodings = tokenizer.encode("Hello, my dog is cute",add_special_tokens = True)

In [ ]:
finetuned_model.cpu()

In [ ]:
input_ids = torch.tensor(encodings).long().unsqueeze(0)
        
outputs = finetuned_model(input_ids,token_type_ids=None)

In [ ]:
filt = [entry for entry in label_list if 3 in entry]